In [2]:
N, M = map(int, input().split())

edge_list = []

for _ in range(M):
    A, B, C = map(int, input().split())
    edge_list.append([A,B,C])

# Bellman-Ford Algorithm
# infty = 70000001 because maximal distance = M * 10000
infty = 70000001
dist = [infty for _ in range(N + 1)]
dist[1] = 0
for _ in range(N -1):
    for edge in edge_list:
        s = edge[0]
        d = edge[1]
        w = edge[2]
        new_dist = dist[s] + w
       
        if dist[s] == infty:  # 실제로 무한이 아니기 때문에 확인해야 한다.
            continue
        if new_dist < dist[d]:
            dist[d] = new_dist
negative_cycle = False
for edge in edge_list:
    s = edge[0]
    d = edge[1]
    w = edge[2]
    if dist[s] == infty:  # 실제로 무한이 아니기 때문에 확인해야 한다.
            continue
    if dist[d] > dist[s] + w:
        negative_cycle = True
        break

# output
if negative_cycle:
    print(-1)
else:
    for i in range(2, N + 1):
        if dist[i] >= infty:
            print(-1)
        else:
            print(dist[i])

 3 4
 1 2 4
 1 3 3
 2 3 -4
 3 1 -2


-1


In [7]:
import heapq

V, E = map(int, input().split())
start_vertex = int(input())

adj_list = [[] for _ in range(V + 1)]

for _ in range(E):
    u,v,w = map(int, input().split())
    adj_list[u].append([v,w])

# Dijkstra
flag = [0] * (V + 1) # 0 = unseen, 1 = fringe, 2 = already done
infty = 10 * 300000 + 1
dist = [infty for _ in range(V + 1)]
dist[start_vertex] = 0
heap = [(0, start_vertex)]

# 일반적인 라이브러리의 힙 자료구조는
# DecreaseKey 연산을 지원하지 않는다.
# 이에 따라 Fringe vertex를 다시 Relaxation하는 경우
# 힙에 그대로 넣어준다.
# 대신 pop을 한 후 확인할 때 해당 정점이 이미 최단 경로가 결정되었는지 확인한다.
while len(heap) > 0:
    u = heapq.heappop(heap)[1]
    if flag[u] == 2:
        continue
    flag[u] = 2
    
    for incident_edge in adj_list[u]:
        adj_vertex = incident_edge[0]
        weight = incident_edge[1]
        if flag[adj_vertex] != 2:
            if dist[adj_vertex] > dist[u] + weight:
                dist[adj_vertex] = dist[u] + weight
                flag[adj_vertex] = 1
                heapq.heappush(heap, (dist[adj_vertex], adj_vertex))

# output
for i in range(1, V+1):
    # 음수 간선이 없으므로 Bellman-Ford Alogirhtm 보다 좀더 느슨하게 무한대 관리
    if dist[i] >= infty:
        print("INF")
    else: print(dist[i])

 5 6
 1
 5 1 1
 1 2 2
 1 3 3
 2 3 4
 2 4 5
 3 4 6


0
2
3
7
INF


In [44]:
import heapq

T = int(input())

position_map = [0, 
                (0,2), (1,2), (2,2),
                (0,1), (1,1), (2,1),
                (0,0), (1,0), (2,0)]
def manhattan_dist(pos1, pos2):
    x_diff = abs(pos1[0] - pos2[0])
    y_diff = abs(pos1[1] - pos2[1])
    return (x_diff + y_diff)
    
def calculate_H(board : str):
    total_manhattan_dist = 0
    for i in range(len(board)):
        if board[i] == '#': continue
        original_pos = position_map[int(board[i])]
        cur_pos = position_map[i + 1]
        total_manhattan_dist += manhattan_dist(original_pos, cur_pos)
    return total_manhattan_dist

def change_board(board : str, blank_pos : int):
    quotient = blank_pos // 3 # 0: 위, 1: 가운데, 2: 아래
    remainder = blank_pos % 3 # 0: 왼쪽, 1: 가운데, 2: 오른쪽
    changable_boards = []
    # 위의 조각과 바꾸기
    if quotient > 0:
        changed_piece = board[blank_pos - 3]
        new_board = board.replace(changed_piece, '#')
        new_board = new_board[:blank_pos] + changed_piece + new_board[blank_pos + 1:]
        changable_boards.append(new_board)
    # 아래의 조각과 바꾸기
    if quotient < 2:
        changed_piece = board[blank_pos + 3]
        new_board = board.replace(changed_piece, '#')
        new_board = new_board[:blank_pos] + changed_piece + new_board[blank_pos + 1:]
        changable_boards.append(new_board)
    # 왼쪽 조각과 바꾸기
    if remainder > 0:
        changed_piece = board[blank_pos - 1]
        new_board = board.replace(changed_piece, '#')
        new_board = new_board[:blank_pos] + changed_piece + new_board[blank_pos + 1:]
        changable_boards.append(new_board)
    if remainder < 2:
        changed_piece = board[blank_pos + 1]
        new_board = board.replace(changed_piece, '#')
        new_board = new_board[:blank_pos] + changed_piece + new_board[blank_pos + 1:]
        changable_boards.append(new_board)
    return changable_boards

# 테스트 케이스가 많으므로 시간 절약용
# board : output
# 특히, 불가능한 경우 굉장히 많은 계산이 필요하다.
memorized_board = {}

def A_star(start_board : str):
    # 이미 완료한 테스트 케이스인지 확인
    if start_board in memorized_board:
        print(memorized_board[start_board])
        return
    
    is_possible = False # 가능/불가능
    g_score_hash = {} # string : g(b) 해시 테이블
    g_score_hash[start_board] = 0
    
    # init
    open_board_heap = [(0 + calculate_H(start_board), start_board)]
    while len(open_board_heap) > 0:
        heap_top = heapq.heappop(open_board_heap)
        cur_board = heap_top[1]
        cur_g_score = g_score_hash[cur_board]
        
        # 정답인 경우
        if calculate_H(cur_board) == 0:
            print(cur_g_score)
            memorized_board[start_board] = cur_g_score
            is_possible = True
            break
        # 불가능이 이미 판단된 경우
        if cur_board in memorized_board:
            if memorized_board[cur_board] == "impossible":
                break
        
        # 그렇지 않은 경우, 빈칸을 기준으로 이동 탐색
        # A* 알고리즘의 슈도 코드는 복잡하지만
        # DecreaseKey가 지원되지 않고, 최단 경로가 고정된 정점도 다시 확인하므로
        # 그냥 힙에 넣는다.
        blank_pos = cur_board.find('#')
        changable_boards = change_board(cur_board, blank_pos)
        for new_board in changable_boards:
            if new_board in g_score_hash:
                if g_score_hash[new_board] > cur_g_score + 1:
                    g_score_hash[new_board] = cur_g_score + 1
                    heapq.heappush(open_board_heap,
                           (g_score_hash[new_board] + calculate_H(new_board), new_board))
            else:
                g_score_hash[new_board] = cur_g_score + 1
                heapq.heappush(open_board_heap,
                           (g_score_hash[new_board] + calculate_H(new_board), new_board))
    
    if not is_possible:
        print("impossible")
        
        # 시간을 줄이기 위해 저장
        memorized_board[start_board] = "impossible"
        key_list = list(g_score_hash.keys())
        for board in key_list:
            memorized_board[board] = "impossible"

    
    
# execute
for _ in range(T):
    input() # 빈줄 제거
    board = ""
    for _ in range(3):
        line = input()
        board += line
    A_star(board)

 2
 
 123
 4#5
 786


2


 
 123
 456
 87#


impossible


KeyboardInterrupt: Interrupted by user

# Week 10 : Floyd-Warshall Algorithm (BOJ 11403)

In [1]:
N = int(input())

transitive_mat = []

for _ in range(N):
    transitive_mat.append(list(map(int, input().split())))

for k in range(N):
    for i in range(N):
        for j in range(N):
            if transitive_mat[i][k] == 1 and transitive_mat[k][j] == 1:
                transitive_mat[i][j] = 1

for i in range(N):
    for j in range(N):
        print(transitive_mat[i][j], end = ' ')
    print('')

 7
 0 0 0 1 0 0 0
 0 0 0 0 0 0 1
 0 0 0 0 0 0 0
 0 0 0 0 1 1 0
 1 0 0 0 0 0 0
 0 0 0 0 0 0 1
 0 0 1 0 0 0 0


1 0 1 1 1 1 1 
0 0 1 0 0 0 1 
0 0 0 0 0 0 0 
1 0 1 1 1 1 1 
1 0 1 1 1 1 1 
0 0 1 0 0 0 1 
0 0 1 0 0 0 0 
